In [1]:
import numpy as np
from data_loader import load_dataset, load_challenge, save_challenge
from torch import nn
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import StandardScaler
from eval_script import compute_revenue

In [2]:
class BirdCrossEntropyLoss(nn.Module):
    def __init__(self, device):
        super(BirdCrossEntropyLoss, self).__init__()
        self.rev_matrix = torch.tensor(np.array(
            [[0.05, -0.2, -0.2, -0.2, -0.2, -0.2, -0.2],
             [-0.25,  1., -0.3, -0.1, -0.1, -0.1, -0.1],
                [-0.02, -0.1,  1., -0.1, -0.1, -0.1, -0.1],
                [-0.25, -0.1, -0.3,  1., -0.1, -0.1, -0.1],
                [-0.25, -0.1, -0.3, -0.1,  1., -0.1, -0.1],
                [-0.25, -0.1, -0.3, -0.1, -0.1,  1., -0.1],
                [-0.25, -0.1, -0.3, -0.1, -0.1, -0.1,  1.]])).to(device)
        self.cet = nn.CrossEntropyLoss(reduction='none').to(device)

    def forward(self, input, target):
        ce_loss = self.cet(input, target)

        input_labels = torch.argmax(input, dim=1)
        target_labels = torch.argmax(target, dim=1)
        wanted = self.rev_matrix[target_labels, target_labels]
        actual = self.rev_matrix[target_labels, input_labels]
        cost = wanted - actual
        custom_loss = (1 + cost) * ce_loss

        return custom_loss.mean()


In [3]:
class BirdDataset(Dataset):
    def __init__(self, birdset, is_train, device, offset, scaler=None):
        self.device = device
        self.offset = offset
        self.data = np.empty((0, 548), dtype=np.float32)
        self.labels = np.empty((0), dtype=int)
        for bird in birdset:
            if is_train:
                self.data = np.concatenate(
                    (self.data, np.concatenate(birdset[bird]['train_features'])))
                self.labels = np.concatenate(
                    (self.labels, np.concatenate(birdset[bird]['train_labels'])))
            else:
                self.data = np.concatenate(
                    (self.data, np.concatenate(birdset[bird]['test_features'])))
                self.labels = np.concatenate(
                    (self.labels, np.concatenate(birdset[bird]['test_labels'])))

        self.data = np.concatenate((np.zeros((offset, 548)), self.data, np.zeros((offset, 548))))

        hot_matrix = np.eye(7)
        self.labels = hot_matrix[self.labels]

        if scaler == None:
            scaler = StandardScaler()
            scaler.fit(self.data)
        self.scaler = scaler

        self.data = torch.tensor(
            self.scaler.transform(self.data), dtype=torch.float32).to(self.device)
        self.labels = torch.tensor(self.labels).to(self.device)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        x = self.data[index:index+self.offset*2+1].unsqueeze(0)
        y = self.labels[index]
        return x, y


In [4]:
def train(dataloader, model, loss_fn, optimizer, progress_steps):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if progress_steps != None and batch % progress_steps == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn, show_progress):
    num_batches = len(dataloader)
    test_loss = 0
    money = 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            money += compute_revenue(torch.argmax(pred, dim=1).cpu(), torch.argmax(y, dim=1).cpu())

    test_loss /= num_batches
    if show_progress:
        print(f"Avg loss: {test_loss:>8f}, Money saved: {money:.2f}$\n")
    return test_loss

def eval(model_fn, train_dataloader, test_dataloader, device, max_epochs):
    model = model_fn(None).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.005)
    loss_fn = BirdCrossEntropyLoss(device)
    lowest_test_loss = np.infty
    stop_criterion = 0
    for t in range(max_epochs):
        if t % 10 == 0:
            print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer, None)
        test_loss = test(test_dataloader, model, loss_fn, True)
        if test_loss > lowest_test_loss:
            stop_criterion += 1
        else:
            lowest_test_loss = test_loss
            stop_criterion = 0
        if stop_criterion >= 2:
            break
    print("Estimated performance:")
    test(test_dataloader, model, loss_fn, True)
    return model


In [5]:
offset = 6

model_fn = lambda _: nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=(3, 1), stride=1), #input = 1*(2*offset+1)*548
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1)), #input = 16*(2*offset+1-2)*548
        nn.Conv2d(16, 32, kernel_size=(3, 1), stride=1), #input = 16*((2*offset+1-2)//2)*548
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=(2, 1), stride=(2, 1)), #input = 32*((2*offset+1-2)//2-2)*548
        nn.Flatten(),
        nn.Linear(32*(((2*offset+1-2)//2-2)//2)*548, 64), #input = 32*(((2*offset+1-2)//2-2)//2)*548
        nn.ReLU(),
        nn.Linear(64, 7)
    )

class PrintDimension(nn.Module):
    def forward(self, x):
        print("Current dimension:", x.shape)
        return x

device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [6]:
dataset = load_dataset()

In [7]:
train_set = BirdDataset(dataset, True, device, offset)
scaler = train_set.scaler
test_set = BirdDataset(dataset, False, device, offset, scaler)
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=True)

In [8]:
model = eval(model_fn, train_loader, test_loader, device, 500)

Epoch 1
-------------------------------
Avg loss: 0.415972, Money saved: 5668.91$

Avg loss: 0.448586, Money saved: 5838.41$

Avg loss: 0.318905, Money saved: 5832.42$

Avg loss: 0.311502, Money saved: 5902.30$

Avg loss: 0.310938, Money saved: 5662.15$

Avg loss: 0.347687, Money saved: 5416.47$

Avg loss: 0.303438, Money saved: 5881.59$

Avg loss: 0.320849, Money saved: 5714.11$

Avg loss: 0.311603, Money saved: 5920.81$

Estimated performance:
Avg loss: 0.311639, Money saved: 5920.81$



In [9]:
torch.save(model.state_dict(), 'Models/cnn')

In [10]:
challenge = load_challenge()
print(challenge.shape)

(16, 3000, 548)


In [11]:
model = model_fn(None)
model.load_state_dict(torch.load('Models/cnn'))
model.eval()

model = model.to(device)
results = torch.Tensor(np.empty((challenge.shape[0], challenge.shape[1]))).to(device)
for i in range(challenge.shape[0]):
    challenge_subset = challenge[i]
    challenge_subset = scaler.transform(challenge_subset)
    size = len(challenge_subset)
    challenge_subset = torch.Tensor(np.concatenate((np.zeros((offset, 548)), challenge_subset, np.zeros((offset, 548))))).unsqueeze(0).unsqueeze(0).to(device)
    for a in range(size):
        tmp = challenge_subset[:, :, a:a+offset*2+1]
        preds = model(tmp).squeeze()
        pred_labels = torch.argmax(preds)
        results[i, a] = pred_labels
    print('substep done')
results = results.int().cpu().numpy()
print(results, results.shape)
save_challenge('cnn', results)

substep done
substep done
substep done
substep done
substep done
substep done
substep done
substep done
substep done
substep done
substep done
substep done
substep done
substep done
substep done
substep done
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 3 3 3]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 6 6 6]
 [0 0 0 ... 0 0 0]] (16, 3000)


In [12]:
print("Maximum money that can be saved:", compute_revenue(test_loader.dataset.labels.argmax(1).cpu(), test_loader.dataset.labels.argmax(1).cpu()), "$")

Maximum money that can be saved: 6897.0 $
